In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
from datetime import datetime

%matplotlib inline

## Accepted Loan Data

In [54]:
#reading the accepted data into pandas
accepted_df = pd.read_csv('combined.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
#dropping policy code with NaN value
accepted_df = accepted_df[accepted_df['policy_code'].notnull()]

#mapping th term column
accepted_df['term'] = accepted_df['term'].map({' 36 months': 36, ' 60 months' : 60})

#int_rate was loaded as an object data type instead of float due to the '%' character. Let's strip that out and convert the column type.
accepted_df['int_rate'] = accepted_df['int_rate'].map(lambda x : x.replace('%', ''))
accepted_df['int_rate'] = accepted_df['int_rate'].map(float)

In [56]:
#threshold of 70% and dropping the ones above that
accepted_df = accepted_df.dropna(axis=1, thresh=0.30 * accepted_df.shape[0])
accepted_df = accepted_df.dropna(axis=0, thresh = 0.30 * (accepted_df.shape[1]))
accepted_df.shape

(1646778, 110)

In [57]:
#Clean and Analyse the slice of data column 1-7

accepted_df.iloc[:5,:7]

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment
0,1077501,5000.0,5000.0,4975.0,36,10.65,162.87
1,1077430,2500.0,2500.0,2500.0,60,15.27,59.83
2,1077175,2400.0,2400.0,2400.0,36,15.96,84.33
3,1076863,10000.0,10000.0,10000.0,36,13.49,339.31
4,1075358,3000.0,3000.0,3000.0,60,12.69,67.79


In [58]:
# drop the record if value is NaN and convert them in suitable types

accepted_df.id = accepted_df.id.dropna()
accepted_df.id = accepted_df.id.astype(int)

accepted_df.loan_amnt = accepted_df.loan_amnt.dropna()
accepted_df.loan_amnt = accepted_df.loan_amnt.astype(int)

accepted_df.funded_amnt = accepted_df.funded_amnt.dropna()
accepted_df.funded_amnt = accepted_df.funded_amnt.astype(int)

accepted_df.funded_amnt_inv = accepted_df.funded_amnt_inv.dropna()
accepted_df.funded_amnt_inv = accepted_df.funded_amnt_inv.astype(int)

#replace missing values for Term with max value
accepted_df.term = accepted_df.term.fillna(int(accepted_df['term'].value_counts().idxmax()))

#replace missing values for Interest Rate with mean value
accepted_df.int_rate = accepted_df.int_rate.fillna(float(accepted_df.int_rate.mean()))

In [59]:
#Clean and Analyse the slice of data column 8-20
accepted_df.iloc[:5,8:20]

,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title
0,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer
1,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike
2,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business
3,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,personel
4,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,Personal


In [60]:
#replace missing values for grade, sub_grade with max value
accepted_df.grade = accepted_df.grade.fillna(int(accepted_df['term'].value_counts().idxmax()))
accepted_df.sub_grade = accepted_df.sub_grade.fillna(int(accepted_df['term'].value_counts().idxmax()))

#replace missing values for emp_title with Not available
accepted_df.emp_title = accepted_df.emp_title.fillna("Not available")

accepted_df.emp_length.replace('n/a', np.nan,inplace=True)
accepted_df.emp_length.fillna(value=0,inplace=True)

#convert categorical value into numerical value
accepted_df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
accepted_df['emp_length'] = accepted_df['emp_length'].astype(int)

#replace missing values for home_ownership with max value
accepted_df.home_ownership = accepted_df.home_ownership.fillna("OTHER")

accepted_df.annual_inc = accepted_df.annual_inc.dropna()
accepted_df.annual_inc = accepted_df.annual_inc.astype(float)


accepted_df.issue_d = accepted_df.issue_d.fillna(accepted_df['issue_d'].value_counts().idxmax())


#replace missing values for loan_status with Not available
accepted_df.loan_status = accepted_df.loan_status.fillna("Not available")

#replace missing values for pymnt_plan with max value
accepted_df.pymnt_plan = accepted_df.pymnt_plan.fillna(accepted_df['pymnt_plan'].value_counts().idxmax())

#replace missing values for url with Not available
accepted_df.url = accepted_df.url.fillna("Not available")

#replace missing values for loan_status with other
accepted_df.purpose = accepted_df.purpose.fillna("other")

accepted_df.title = accepted_df.title.fillna("Not available")



In [61]:
#Clean and Analyse the slice of data column 21-30
accepted_df.iloc[:5,21:30]

,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc
0,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,3.0
1,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,3.0
2,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,NaN,2.0
3,CA,20.00,0.0,Feb-1996,690.0,694.0,1.0,35.0,10.0
4,OR,17.94,0.0,Jan-1996,695.0,699.0,0.0,38.0,15.0


In [62]:
accepted_df.addr_state = accepted_df.addr_state.fillna("XX")

#drop the record if the value of dti is missing 
accepted_df.dti = accepted_df.dti.dropna()
accepted_df.dti = accepted_df.dti.astype(float)

#replace missing values for delinq_2yrs with max value count
accepted_df.delinq_2yrs = accepted_df.delinq_2yrs.fillna(accepted_df['delinq_2yrs'].value_counts().idxmax()).astype(int)


#fico_range_low & fico_range_high scores should be converted to its mean and range should be above 330 and dropping the NaNs

accepted_df = accepted_df[accepted_df['fico_range_low'] > 330]

accepted_df = accepted_df[accepted_df['fico_range_low'].notnull()]
accepted_df = accepted_df[accepted_df['fico_range_high'].notnull()]

accepted_df['risk_score'] = (accepted_df['fico_range_low'] + accepted_df['fico_range_high']) / 2


#replace missing values for mths_since_last_delinq with 0
accepted_df.mths_since_last_delinq = accepted_df.mths_since_last_delinq.fillna(0).astype(int)

#replace missing values for open account with 0
accepted_df.open_acc = accepted_df.open_acc.fillna(accepted_df['open_acc'].mean()).astype(int)

In [63]:
#Clean and Analyse the slice of data column 31-50
accepted_df.iloc[:5,31:50]

,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low
0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,NaN,Oct-2017,739.0,735.0
1,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0
2,2956.0,98.5%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,NaN,Jun-2017,739.0,735.0
3,5598.0,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,NaN,Apr-2016,604.0,600.0
4,27783.0,53.9%,38.0,f,0.0,0.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-2017,67.30,NaN,Jan-2017,694.0,690.0


In [64]:

accepted_df.revol_bal = accepted_df.revol_bal.dropna().astype(int)

accepted_df.revol_util = accepted_df.revol_util.dropna()

accepted_df.total_acc = accepted_df.total_acc.fillna(accepted_df['total_acc'].mean()).astype(int)

#replace missing values for initial_list_status with max value
accepted_df.initial_list_status = accepted_df.initial_list_status.fillna(accepted_df['initial_list_status'].value_counts().idxmax())

#replace missing values for out_prncp with max value
accepted_df.out_prncp = accepted_df.out_prncp.fillna(accepted_df['out_prncp'].value_counts().idxmax()).astype(int)



#math.ceil(i*100)/100
ceil_function= lambda x: math.ceil(x*100)/100

#replace missing values for out_prncp_inv with max value
accepted_df.out_prncp_inv = accepted_df.out_prncp_inv.fillna(accepted_df['out_prncp_inv'].value_counts().idxmax()).astype(int)

#replace missing values for out_prncp_inv with max value
accepted_df.total_pymnt = accepted_df.total_pymnt.fillna(0).astype(float)
accepted_df.total_rec_prncp = accepted_df.total_rec_prncp.fillna(0).astype(float)
accepted_df.total_rec_late_fee = accepted_df.total_rec_late_fee.fillna(0).astype(float)
accepted_df.recoveries = accepted_df.recoveries.fillna(0).astype(float)
accepted_df.collection_recovery_fee = accepted_df.collection_recovery_fee.fillna(0).astype(float)

accepted_df['total_pymnt'] = accepted_df['total_pymnt'].apply(ceil_function)
accepted_df['total_rec_prncp'] = accepted_df['total_rec_prncp'].apply(ceil_function)
accepted_df['total_rec_late_fee'] = accepted_df['total_rec_late_fee'].apply(ceil_function)
accepted_df['recoveries'] = accepted_df['recoveries'].apply(ceil_function)
accepted_df['collection_recovery_fee'] = accepted_df['collection_recovery_fee'].apply(ceil_function)

In [65]:
#Clean and Analyse the slice of data column 51-70
accepted_df.iloc[:5,50:70]

,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl
0,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#application type, acc_now_delinq, chargeoff_within_12_mnths, delinq_amnt, pub_rec, tax_lines
accepted_df.application_type = accepted_df.application_type.fillna((accepted_df['application_type'].value_counts().idxmax()))

accepted_df.acc_now_delinq = accepted_df.acc_now_delinq.fillna((accepted_df['acc_now_delinq'].value_counts().idxmax())).astype(int)

accepted_df.chargeoff_within_12_mths = accepted_df.chargeoff_within_12_mths.fillna((accepted_df['chargeoff_within_12_mths'].value_counts().idxmax())).astype(int)

accepted_df.delinq_amnt = accepted_df.delinq_amnt.fillna((accepted_df['delinq_amnt'].value_counts().idxmax())).astype(int)

accepted_df.pub_rec_bankruptcies = accepted_df.pub_rec_bankruptcies.fillna((accepted_df['pub_rec_bankruptcies'].value_counts().idxmax())).astype(int)

accepted_df.tax_liens = accepted_df.tax_liens.fillna((accepted_df['tax_liens'].value_counts().idxmax())).astype(int)

In [67]:
accepted_df.shape

(1646778, 111)

## Declined Loan Data


In [68]:
#reading the data into pandas
rejected_df = pd.read_csv('outputRejectStats.csv')

In [69]:
rejected_df.shape

(16131472, 9)

In [71]:
rejected_df.iloc[:3,:9]

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
